## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-08-54-54 +0000,wapo,U.S. and China likely to extend trade truce to...,https://www.washingtonpost.com/world/2025/07/2...
1,2025-07-28-08-51-33 +0000,nyt,"Trump and Starmer Expected to Discuss Trade, G...",https://www.nytimes.com/2025/07/28/us/politics...
2,2025-07-28-08-42-00 +0000,wsj,Global Markets Rise After U.S. and EU Reach Deal,https://www.wsj.com/finance/global-markets-ris...
3,2025-07-28-08-25-40 +0000,wapo,Walmart stabbing suspect faces terrorism and a...,https://www.washingtonpost.com/nation/2025/07/...
4,2025-07-28-08-25-15 +0000,nypost,"Epstein furor undermines public trust, Republi...",https://nypost.com/2025/07/28/us-news/epstein-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2367/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,30
13,gaza,14
3,trade,13
22,deal,12
532,aid,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...,76
138,2025-07-27-17-13-12 +0000,nypost,Trump says ‘Israel is gonna have to make a dec...,https://nypost.com/2025/07/27/us-news/trump-sa...,71
8,2025-07-28-08-12-00 +0000,wsj,Stock Market Today: Dow Futures Rise After Tru...,https://www.wsj.com/livecoverage/stock-market-...,71
1,2025-07-28-08-51-33 +0000,nyt,"Trump and Starmer Expected to Discuss Trade, G...",https://www.nytimes.com/2025/07/28/us/politics...,70
164,2025-07-27-12-10-56 +0000,nypost,Trump and Ursula von der Leyen to meet for hig...,https://nypost.com/2025/07/27/world-news/trump...,68


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,76,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...
186,45,2025-07-27-09-02-00 +0000,wsj,How Food Supplies in Gaza Have Dwindled Under ...,https://www.wsj.com/world/middle-east/gaza-pal...
164,38,2025-07-27-12-10-56 +0000,nypost,Trump and Ursula von der Leyen to meet for hig...,https://nypost.com/2025/07/27/world-news/trump...
124,29,2025-07-27-18-40-18 +0000,wapo,"Tom Lehrer, master satirist of Cold War era, d...",https://www.washingtonpost.com/obituaries/2025...
81,27,2025-07-27-22-28-40 +0000,nypost,AOC ripped as her NYC district is overrun by s...,https://nypost.com/2025/07/27/us-news/aoc-ripp...
90,26,2025-07-27-22-10-11 +0000,bbc,Suspect faces terrorism charges in stabbing of...,https://www.bbc.com/news/articles/ckgjmr9y7x2o
33,25,2025-07-28-04-03-34 +0000,nypost,Cedar Point’s ‘Siren’s Curse’ rollercoaster br...,https://nypost.com/2025/07/28/us-news/cedar-po...
103,25,2025-07-27-20-58-27 +0000,nyt,3 Dead After Small Plane Crashes Off Californi...,https://www.nytimes.com/2025/07/27/us/plane-cr...
72,25,2025-07-27-23-05-59 +0000,nypost,Viral images of starving Gaza boy don’t tell t...,https://nypost.com/2025/07/27/world-news/viral...
165,25,2025-07-27-12-09-57 +0000,bbc,Thailand and Cambodia agree to talks in Malays...,https://www.bbc.com/news/articles/cy854585r32o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
